# Statystyczne metody przetwarzania danych 

## Laboratorium 3 - algorytm Najbliższej Średniej (NM)


### Opis
Celem laboratorium jest implementacja klasyfikatora najbliższej średniej NM (*Nearest Mean*).

### Zbiór danych

Zbiór danych znajduje się w `dataset/leaf.csv`. Jest to zbiór danych pobrany z adresu: <https://archive.ics.uci.edu/ml/datasets/leaf>.

### Przesyłanie zadań

Wszystkie pliki należy spakować archiwizatorem **zip** i przesłać za pośrednictwem platformy WIKAMP. Poniżej oczekiwana zawartość archiwum:

```
+-- 📂 [IMIE I NAZWISKO].zip
    +-- 📜 Lab03.ipynb
    +-- 📂 dataset
        +-- 📜 leaf.csv
        +-- 📜 ReadMe.pdf
```

*UWAGA: Wysyłając zadanie potwierdasz, że wykonałeś je samodzielnie i jest to Twoja indywidualna praca i materiał przedstawiony w tej pracy jest dla Ciebie zrozumiały.*

### Zadanie

Należy wykonać następujące czynności w celu realizacji niniejszego zadania:
* Wczytaj dane.
* Rozszerz zbiór danych.
* Podziel dane na zbiór treningowy i testowy.
* Zaimplementuj funkcję, która zwraca macierz kowariancji (*uwaga: biblioteka `numpy` posiada gotową implementację `cov` z którą możesz porównać wynik*).
* **Zaimplementuj klasyfikator najbliższej średniej (NM) z zastosowaniem odległości Euklidesa**.
* **Zaimplementuj klasyfikator najbliższej średniej (NM) z zastosowaniem odległości Machalanobisa**.
* Opisz wyniki klasyfikatorów i porównaj je z klasyfikatorem *k*NN (porównaj w kontekście różnych metryk - obowiązkowo tablica pomyłek).

> Podpowiedź 1: Do obliczenia macierzy odwrotnej możesz użyć gotową implementację, np. funkcję `linalg.inv` z biblioteki `numpy`.

> Podpowiedź 2: Do wszelkich podstawowych operacji na macierzach (mnożenie, transpozycja, dodawanie, odejmowanie, itp.) możesz zastosować gotową implementację, np. bibliotekę `numpy`.

> UWAGA 1: W niniejszym zadaniu jest dowolność implementacji (nie trzeba trzymać się struktury z poprzedniego zadania), jednak algorytm NM należy zaimplementować samodzielnie bez korzystania z istniających rozwiązań (jak np. z biblioteki `scikit-learn`).

> UWAGA 2: Wszystkie wykonane elementy zadania powinny posiadać stosowne komentarze i opisy.


<span style="text-decoration:underline">Referencje</span>

1. Mahalanobis, P C, _On test and measures of group divergence : theoretical formulae_, Journal and Proceedings of Asiatic Society of Bengal (New Series) Vol. 26, pp. 541-588. 1930. (URL: http://library.isical.ac.in:8080/xmlui/bitstream/handle/10263/1639/029.pdf)
2. McLachlan, Goeffrey J. _Mahalanobis distance_, Resonance, pp. 20-26. 1999. (URL: https://www.ias.ac.in/article/fulltext/reso/004/06/0020-0026)

### PONIŻEJ WYKONAJ ZADANIE

In [3]:
import numpy as np
import math
from sklearn.metrics import confusion_matrix
from sklearn.metrics import balanced_accuracy_score

with open('./dataset/leaf.csv', 'rb') as f:
    data = np.genfromtxt(f, delimiter=',')

five_classes = [3., 5., 6., 7., 8.]
twelve_classes = [3., 5., 6., 7., 8., 10. , 11., 12., 13., 14., 15., 30.]

def prepare_data_sets(classes):
    test = None
    train = None
    for c in classes:
        filtered = data[(data[:, 0] == c)]
        test_len = round(0.2 * len(filtered))
        train_len = len(filtered) - test_len
        test_c = filtered[:test_len, :]
        train_c = filtered[-train_len:, :]
        if test is None:
            test = test_c
        else:
            test = np.concatenate((test, test_c))
        if train is None:
            train = train_c
        else:
            train = np.concatenate((train, train_c))
    return train, test

train_five_classes, test_five_classes =  prepare_data_sets(five_classes)
train_twelve_classes, test_twelve_classes = prepare_data_sets(twelve_classes)

class TestSample:

    classified_as = None
    original_classification = None

    def __init__(self, original_classification, classified_as):
        self.original_classification = original_classification
        self.classified_as = classified_as

    def is_properly_classified(self):
        return self.original_classification == self.classified_as

class Neighbour:

    distance = None
    class_value = None

    def __init__(self, class_value, distance):
        self.class_value = class_value
        self.distance = distance

def mahalanobis_dist(sample_vector, mean_vector, inverted_cov_matrix):
    subtracted = np.subtract(sample_vector, mean_vector)
    subtracted_transposed = np.transpose(subtracted)
    matrix = np.matmul(subtracted_transposed, inverted_cov_matrix)
    return np.vdot(matrix, subtracted)

def euclidian_dist(sample1, sample2, features_list):
    value = 0
    for feature in features_list:
        value += (sample1[feature] - sample2[feature]) ** 2
    return math.sqrt(value)

def get_class_from_nearest_neighbours(nearest_neighbours):
    nearest_neighbours_dict = {}
    for nearest_neighbour in nearest_neighbours:
        if nearest_neighbour.class_value in nearest_neighbours_dict:
            nearest_neighbours_dict[nearest_neighbour.class_value].append(nearest_neighbour)
        else:
            nearest_neighbours_dict[nearest_neighbour.class_value] = [nearest_neighbour]
    max_size = 0
    class_value = None
    for k in nearest_neighbours_dict.keys():
        size = len(nearest_neighbours_dict[k])
        if size > max_size:
            max_size = size
            class_value = k
    return class_value

def get_covariant_matrix(class_samples, mean_vector,  features_list):
    class_vectors = []
    for train_sample in class_samples:
        train_sample_vector = get_vector_from_sample(train_sample, features_list)
        class_vectors.append(train_sample_vector)
    mean_vector_len = len(class_vectors)
    class_vectors = np.transpose(np.asarray(class_vectors))
    mean_vectors = []
    for i in range(mean_vector_len):
        mean_vectors.append(mean_vector)
    mean_vectors = np.transpose(np.asarray(mean_vectors))
    subtracted = np.subtract(class_vectors, mean_vectors)
    transposed_subtracted = np.transpose(subtracted)
    return (1 / len(class_vectors)) * np.matmul(subtracted, transposed_subtracted)

def get_mean_vector(filtered, features_list):
    mean = []
    for i in range(len(features_list)):
        mean.append(0.)
    for sample in filtered:
        sample_vector = get_vector_from_sample(sample, features_list)
        for i in range(len(sample_vector)):
            mean[i] += sample_vector[i]
    for i in range(len(features_list)):
        mean[i] /= len(filtered)
    return np.asarray(mean)

def get_vector_from_sample(sample, features_list):
    vector = []
    for feature in features_list:
        vector.append(sample[feature])
    return np.asarray(vector)

def get_mean_value(filtered, features_list):
    mean = []
    for i in range(16):
        mean.append(0.)
    for feature in features_list:
        for sample in filtered:
            mean[feature] += sample[feature]
        mean[feature] /= len(filtered)
    return np.asarray(mean)

def get_mean_values(train_set, classes, features_list):
    mean_values = {}
    for c in classes:
        filtered = train_set[(train_set[:, 0] == c)]
        mean_values[c] = get_mean_value(filtered, features_list)
    return mean_values

classified_samples = []
def classify_knn_samples(test_set, train_set, k, features_list):
    classified_samples.clear()
    for test_sample in test_set:
        neighbours = []
        for train_sample in train_set:
            dist = euclidian_dist(test_sample, train_sample, features_list)
            neighbours.append(Neighbour(train_sample[0], dist))
        neighbours.sort(key=lambda x: x.distance, reverse=False)
        nearest_neighbours = neighbours[:k]
        classified_class = get_class_from_nearest_neighbours(nearest_neighbours)
        classified_samples.append(TestSample(test_sample[0], classified_class))

def classify_nm_euclides_samples(test_set, train_set, classes, features_list):
    classified_samples.clear()
    mean_values = get_mean_values(train_set, classes, features_list)
    for test_sample in test_set:
        min_dist = None
        classified_class = None
        for c in mean_values.keys():
            dist = euclidian_dist(test_sample, mean_values[c], features_list)
            if min_dist is None or dist < min_dist:
                min_dist = dist
                classified_class = c
        classified_samples.append(TestSample(test_sample[0], classified_class))

def classify_nm_mahalanobis_samples(test_set, train_set, classes, features_list):
    classified_samples.clear()
    inv_cov_matrices = {}
    mean_vectors = {}
    for c in classes:
        class_samples = train_set[(train_set[:, 0] == c)]
        mean_vector = get_mean_vector(class_samples, features_list)
        mean_vectors[c] = mean_vector
        cov = get_covariant_matrix(class_samples, mean_vector, features_list)
        inv_cov_matrices[c] = np.linalg.inv(cov)
    for test_sample in test_set:
        test_sample_vector = get_vector_from_sample(test_sample, features_list)
        min_dist = None
        classified_class = None
        for c in inv_cov_matrices.keys():
            dist = mahalanobis_dist(test_sample_vector, mean_vectors[c], inv_cov_matrices[c])
            if min_dist is None or dist < min_dist:
                min_dist = dist
                classified_class = c
        classified_samples.append(TestSample(test_sample[0], classified_class))


def print_accuracy():
    properly_classified_samples = 0
    for classified_sample in classified_samples:
        if classified_sample.is_properly_classified():
            properly_classified_samples += 1
    print("Accuracy is: " + str(properly_classified_samples / len(classified_samples)))

def fill_classification_data(classified, class_true, class_pred):
    for sample in classified:
        class_true.append(sample.original_classification)
        class_pred.append(sample.classified_as)

def print_confusion_matrix(classified):
    class_true = []
    class_pred = []
    fill_classification_data(classified, class_true, class_pred)
    print("Confusion matrix: ")
    print(confusion_matrix(class_true, class_pred))

def print_balanced_accuracy(classified):
    class_true = []
    class_pred = []
    fill_classification_data(classified, class_true, class_pred)
    print("Balanced accuracy is: " + str(balanced_accuracy_score(class_true, class_pred)))

def print_classification_parameters(classified):
    print_accuracy()
    print_balanced_accuracy(classified)
    print_confusion_matrix(classified)

print("5 classes, features {4, 12, 15}, k 3")
features = [4, 12, 15]
classify_knn_samples(test_five_classes, train_five_classes, 3, features)
print("kNN:")
print_classification_parameters(classified_samples)
classify_nm_euclides_samples(test_five_classes, train_five_classes, five_classes, features)
print("NM euclides:")
print_classification_parameters(classified_samples)
classify_nm_mahalanobis_samples(test_five_classes, train_five_classes, five_classes, features)
print("NM mahalanobis:")
print_classification_parameters(classified_samples)
# Results:
# For classes {3, 5, 6, 7, 8} features {4, 12, 15} and k equals to 3 we
# have the following results:
#
# 1. kNN:
#   Accuracy is: 0.8
#   Balanced accuracy is: 0.8
# 2. NM euclides:
#   Accuracy is: 0.5
#   Balanced accuracy is: 0.5
# 3. NM mahalanobis:
#   Accuracy is: 0.9
#   Balanced accuracy is: 0.9
#
#  As one can see, the 'NM mahalanobis' method has the best results comparing to 'kNN' and 'NM euclides' ones

print('-----------------------------------------')
print("12 classes, features {4, 5, 6, 12, 15}, k 3")
features = [4, 5, 6, 12, 15]
classify_knn_samples(test_twelve_classes, train_twelve_classes, 3, features)
print("kNN:")
print_classification_parameters(classified_samples)
classify_nm_euclides_samples(test_twelve_classes, train_twelve_classes, twelve_classes, features)
print("NM euclides:")
print_classification_parameters(classified_samples)
classify_nm_mahalanobis_samples(test_twelve_classes, train_twelve_classes, twelve_classes, features)
print("NM mahalanobis:")
print_classification_parameters(classified_samples)
# Results:
# For classes {3, 5, 6, 7, 8, 10, 11, 12, 13, 14, 15, 30} features {4, 5, 6, 12, 15} and k equals to 3 we
# have the following results:
#
# 1. kNN:
#   Accuracy is: 0.5555555555555556
#   Balanced accuracy is: 0.5555555555555555
# 2. NM euclides:
#   Accuracy is: 0.5925925925925926
#   Balanced accuracy is: 0.5972222222222222
# 3. NM mahalanobis:
#   Accuracy is: 0.8518518518518519
#   Balanced accuracy is: 0.8333333333333334
#
#  As one can see, the 'NM mahalanobis' method has again the best results comparing to 'kNN' and 'NM euclides' ones


5 classes, features {4, 12, 15}, k 3
kNN:
Accuracy is: 0.8
Balanced accuracy is: 0.8
Confusion matrix: 
[[1 0 0 1 0]
 [0 2 0 0 0]
 [0 0 1 0 1]
 [0 0 0 2 0]
 [0 0 0 0 2]]
NM euclides:
Accuracy is: 0.5
Balanced accuracy is: 0.5
Confusion matrix: 
[[0 1 1 0 0]
 [0 0 0 2 0]
 [0 0 2 0 0]
 [0 0 1 1 0]
 [0 0 0 0 2]]
NM mahalanobis:
Accuracy is: 0.9
Balanced accuracy is: 0.9
Confusion matrix: 
[[2 0 0 0 0]
 [0 1 1 0 0]
 [0 0 2 0 0]
 [0 0 0 2 0]
 [0 0 0 0 2]]
-----------------------------------------
12 classes, features {4, 5, 6, 12, 15}, k 3
kNN:
Accuracy is: 0.5555555555555556
Balanced accuracy is: 0.5555555555555555
Confusion matrix: 
[[1 0 0 0 0 0 0 0 0 0 0 1]
 [0 2 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 1 0 0 0 0 0]
 [0 0 0 2 0 0 0 0 0 0 0 0]
 [0 0 0 0 2 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 1 0 0 1]
 [0 0 0 0 0 0 3 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 1 0 0]
 [1 0 0 0 0 0 0 0 1 0 0 1]
 [0 0 0 0 0 0 0 2 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 2 0 0 0 0 0 0]]
NM euclides:
Accuracy is: 0.592592592

0 2 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 1]
 [0 0 0 0 0 0 0 0 3 0 0 0]
 [0 0 0 0 0 0 0